In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Tugas Akhir/gan

/content/gdrive/MyDrive/Tugas Akhir/gan


In [3]:
from numpy import expand_dims, zeros, ones, asarray
from numpy.random import randn, randint

from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Model, Sequential
import tensorflow as tf

from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, Concatenate
from keras.layers import LeakyReLU, Dropout, Lambda, Activation, BatchNormalization
from keras.applications.mobilenet_v2 import MobileNetV2

#from keras.utils import to_categorical

from matplotlib import pyplot as plt
from keras import backend as K
import numpy as np

In [36]:
def encoder(in_shape=(28,28,1), latent_dim=100):
    in_image = Input(shape=in_shape)
    X = Conv2D(32, (3,3), strides=(2,2), padding='same')(in_image)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(64, (3,3), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(128, (3,3), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Flatten()(X)
    X = Dropout(0.4)(X) #Consider adding more dropout layers to minimize overfitting - remember we work with limited labeled data. 
    X = Dense(latent_dim)(X)
    
    model = Model(inputs=in_image, outputs=X)
    
    return model

In [37]:
enc_model=encoder()
print(enc_model.summary())

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 14, 14, 32)        0         
                                                                 
 conv2d_19 (Conv2D)          (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 7, 7, 64)          0         
                                                                 
 conv2d_20 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_29 (LeakyReLU)  (None, 4, 4, 128)         0  

In [38]:
def decoder(latent_dim = 100):
	
	in_lat = Input(shape=(latent_dim,))
	#Start with enough dense nodes to be reshaped and ConvTransposed to 28x28x1
	n_nodes = 256 * 7 * 7
	X = Dense(n_nodes)(in_lat)
	X = LeakyReLU(alpha=0.2)(X)
	X = Reshape((7, 7, 256))(X)
	
	X = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same')(X) #14x14x128
	X = LeakyReLU(alpha=0.2)(X)
	
	X = Conv2DTranspose(64, (3,3), strides=(1,1), padding='same')(X) #14x14x64
	X = LeakyReLU(alpha=0.2)(X)
	# output
	out_layer = Conv2DTranspose(1, (3,3), strides=(2,2), activation='tanh', 
                             padding='same')(X) #28x28x1
	# define model
	model = Model(in_lat, out_layer)
	return model


In [39]:
dec_model=decoder()
print(dec_model.summary())

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 100)]             0         
                                                                 
 dense_8 (Dense)             (None, 12544)             1266944   
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 12544)             0         
                                                                 
 reshape_3 (Reshape)         (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose_9 (Conv2DT  (None, 14, 14, 128)      295040    
 ranspose)                                                       
                                                                 
 leaky_re_lu_31 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                          

In [40]:
def define_generator(enc_model, dec_model):
    # dec_model.trainable = False
    gen_output = dec_model(enc_model.output) #Gen. output is the input to disc. 
    model = Model(enc_model.input, gen_output)
    # model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

In [41]:
gen_model = define_generator(enc_model, dec_model)
print(gen_model.summary())

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 14, 14, 32)        0         
                                                                 
 conv2d_19 (Conv2D)          (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 7, 7, 64)          0         
                                                                 
 conv2d_20 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_29 (LeakyReLU)  (None, 4, 4, 128)         0  

In [42]:
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    in_image = Input(shape=in_shape)
    X = Conv2D(32, (3,3), strides=(2,2), padding='same')(in_image)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(64, (3,3), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(128, (3,3), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Flatten()(X)
    X = Dropout(0.4)(X) #Consider adding more dropout layers to minimize overfitting - remember we work with limited labeled data. 
    X = Dense(n_classes)(X)
    
    model = Model(inputs=in_image, outputs=X)
    
    return model

In [43]:
disc_model=define_discriminator()
print(disc_model.summary())

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 14, 14, 32)        0         
                                                                 
 conv2d_22 (Conv2D)          (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_34 (LeakyReLU)  (None, 7, 7, 64)          0         
                                                                 
 conv2d_23 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_35 (LeakyReLU)  (None, 4, 4, 128)         0  

In [44]:
def define_gan(gen_model, disc_model):
	
    disc_model.trainable = False # make unsup. discriminator not trainable
    gan_output = disc_model(gen_model.output) #Gen. output is the input to disc. 
    model = Model(gen_model.input, gan_output)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

In [45]:
gan_model = define_gan(gen_model, disc_model)
print(gan_model.summary())

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 14, 14, 32)        0         
                                                                 
 conv2d_19 (Conv2D)          (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 7, 7, 64)          0         
                                                                 
 conv2d_20 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_29 (LeakyReLU)  (None, 4, 4, 128)         0  

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# from tensorflow import keras as k

# n_channels = 3
# latent_space_dimension: int = 200
# l2_penalty: float = 0.0
# KERNEL_INITIALIZER = k.initializers.RandomNormal(mean=0.0, stddev=0.02)
# ALMOST_ONE = k.initializers.RandomNormal(mean=1.0, stddev=0.02)

# model = k.models.Sequential([
#                       k.layers.Input(shape=(28, 28, n_channels)),
#                       k.layers.Conv2D(64,
#                                       (4, 4),
#                                       kernel_initializer=KERNEL_INITIALIZER,
#                                       strides=(2, 2),
#                                       padding="same",
#                                       use_bias=False,
#                                       kernel_regularizer=k.regularizers.l2(l2_penalty)),
#                       k.layers.LeakyReLU(0.2),
#                       k.layers.Conv2D(128,
#                                       (4, 4),
#                                       kernel_initializer=KERNEL_INITIALIZER,
#                                       strides=(2, 2),
#                                       padding="same",
#                                       use_bias=False,
#                                       kernel_regularizer=k.regularizers.l2(l2_penalty)),
#                       k.layers.BatchNormalization(
#                           beta_initializer=ALMOST_ONE,
#                           gamma_initializer=ALMOST_ONE,
#                           momentum=0.1,
#                           epsilon=1e-5),
#                       k.layers.LeakyReLU(0.2),
#                       k.layers.Conv2D(256,
#                                       (4, 4),
#                                       kernel_initializer=KERNEL_INITIALIZER,
#                                       strides=(2, 2),
#                                       padding="same",
#                                       use_bias=False,
#                                       kernel_regularizer=k.regularizers.l2(l2_penalty)),
#                       k.layers.BatchNormalization(
#                           beta_initializer=ALMOST_ONE,
#                           gamma_initializer=ALMOST_ONE,
#                           momentum=0.1,
#                           epsilon=1e-5),
#                       k.layers.LeakyReLU(0.2),
#                       k.layers.Conv2D(512,
#                                       (4, 4),
#                                       kernel_initializer=KERNEL_INITIALIZER,
#                                       strides=(2, 2),
#                                       padding="same",
#                                       use_bias=False,
#                                       kernel_regularizer=k.regularizers.l2(l2_penalty)),
#                       k.layers.BatchNormalization(
#                           beta_initializer=ALMOST_ONE,
#                           gamma_initializer=ALMOST_ONE,
#                           momentum=0.1,
#                           epsilon=1e-5),
#                       k.layers.LeakyReLU(0.2),
#                       k.layers.Conv2D(latent_space_dimension,
#                                       (4, 4),
#                                       kernel_initializer=KERNEL_INITIALIZER,
#                                       strides=(2, 2),
#                                       padding="same",
#                                       use_bias=False,
#                                       kernel_regularizer=k.regularizers.l2(l2_penalty)),
#                       k.layers.BatchNormalization(
#                           beta_initializer=ALMOST_ONE,
#                           gamma_initializer=ALMOST_ONE,
#                           momentum=0.1,
#                           epsilon=1e-5),
#                       k.layers.Flatten()

                
# ])